In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import re
import nltk
import datetime
import os
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')
stemmer = SnowballStemmer("english")
stops = set(stopwords.words("english"))

# from google.colab import files
# uploaded = files.upload()

from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
basedir = '/content/drive/My Drive/Colab Notebooks/Numeraxial/'
os.chdir(basedir)
filename = 'LM_dict.csv' # LM dictionary
# landm = pd.read_csv(os.path.join(basedir, filename),header=0, index_col='Word')
landm = pd.read_csv(filename,header=0, index_col='Word')

filename = 'affectivespace.csv'
aff = pd.read_csv(filename,header=None, index_col=0)
aff.columns = pd.Index(np.arange(0,100))
aff.index.name = 'Word'

In [0]:
# Get the path of current working directory 
path = os.getcwd() 
  
# Get the list of all files and directories 
# in current working directory 
dir_list = os.listdir(path) 
  
print("Files and directories in '", path, "' :")  
# print the list 
print(dir_list)

idx_pickle=[]
idx_csv=[]
list_all = os.listdir(os.path.join(path,'data'))
for num, lst in enumerate(list_all):
  if re.findall(r".pickle", lst) != []:
    idx_pickle.append(num)
  elif re.findall(r".csv", lst) != []:
    idx_csv.append(num)

pickle_set = []
csv_set = []
for i in idx_pickle:
  pickle_set.append(list_all[i])
for i in idx_csv:
  csv_set.append(list_all[i])

pickle_set.sort()
csv_set.sort()
print(pickle_set)
print(csv_set)
n = len(pickle_set)

Files and directories in ' /content/drive/My Drive/Colab Notebooks/Numeraxial ' :
['affectivespace.csv', 'data', 'LM_dict.csv', 'NASDAQ.ipynb', 'Data_gen.ipynb', 'data_cleaned']
['news_AAPL_dict.pickle', 'news_AXP_dict.pickle', 'news_BA_dict.pickle', 'news_CAT_dict.pickle', 'news_CSCO_dict.pickle', 'news_CVX_dict.pickle', 'news_DIS_dict.pickle', 'news_GE_dict.pickle', 'news_GS_dict.pickle', 'news_HD_dict.pickle', 'news_IBM_dict.pickle', 'news_INTC_dict.pickle', 'news_JNJ_dict.pickle', 'news_JPM_dict.pickle', 'news_KO_dict.pickle', 'news_MCD_dict.pickle', 'news_MMM_dict.pickle', 'news_MRK_dict.pickle', 'news_MSFT_dict.pickle', 'news_NKE_dict.pickle', 'news_PFE_dict.pickle', 'news_PG_dict.pickle', 'news_TRV_dict.pickle', 'news_UNH_dict.pickle', 'news_UTX_dict.pickle', 'news_VZ_dict.pickle', 'news_V_dict.pickle', 'news_WMT_dict.pickle', 'news_XOM_dict.pickle']
['AAPL_20170102_20200529.csv', 'AXP_20170102_20200529.csv', 'BA_20170102_20200529.csv', 'CAT_20170102_20200529.csv', 'CSCO_2017010

In [0]:
def create_dictionary(clean_word_list):
    word_count = {}
    for word in clean_word_list:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1
    return word_count

def findlist(your_list):
  l = [ind for ind in your_list if ind in aff.index]
  return l

In [0]:
for lp in range(n):
  
  filename_pkl = pickle_set[lp]
  with open(os.path.join('data', filename_pkl),'rb') as handle:
    b=pickle.load(handle)

  tkr=[]
  pub_date=[]
  content=[]

  for k, v in b.items():
    tkr.append(v['TICKER'])
    pub_date.append(v['PUBLICATION_DATE'])
    content.append(v['SUMMARY'])

  df_pickle=pd.DataFrame({'Ticker':tkr,'Date':pub_date,'Summary':content})
  df_pickle['Date'] = pd.to_datetime(df_pickle['Date'], format = '%Y-%m-%d %H:%M:%S%z').dt.tz_convert('US/Eastern')


  fullnews=[]
  stemmer = SnowballStemmer("english")
  stops = set(stopwords.words("english"))

  for review in df_pickle['Summary']:
    # review = val['SUMMARY']
    tokens = nltk.word_tokenize(str(review).lower())
    token_words = [w for w in tokens if w.isalpha()]
    # stemmed_words = [stemmer.stem(w) for w in token_words]
    meaningful_words = [w for w in token_words if not w in stops]
    fullnews.append(meaningful_words)

  embedded = []
  for i in range(len(fullnews)):
    dict_words = create_dictionary(fullnews[i])
    wordsinAS = findlist(fullnews[i])
    
    final={}
    for _, key in enumerate(wordsinAS):
      final[key] = dict_words[key]
      
    summingup = 0.0
    for k, v in final.items():
      value = aff.loc[k] * v
      summingup += value.values
      
    if type(summingup) == float:
      embedded.append(np.zeros(100))
    else:
      embedded.append(summingup)#.tolist())
  
  df_final = pd.DataFrame(embedded)
  df_final['TICKER'] = b[0]['TICKER']

  print('Stemming and tokenizing is complete!')

  df_pickle['Date'] = pd.to_datetime(df_pickle['Date'], format = '%Y-%m-%d %H:%M:%S%z').dt.tz_convert('US/Eastern')
  df_final = pd.concat([df_pickle, df_final], axis=1)
  df_final.drop(['TICKER'], axis=1, inplace=True)
  df_final = df_final.set_index('Date').resample('15min').mean().dropna(axis = 'index', how = 'all').reset_index()
  df_final['Ticker'] = df_pickle['Ticker'][0]
  # Reoder columns
  cols = df_final.columns.to_list()[-1:] + df_final.columns.to_list()[:-1]
  df_final = df_final[cols]


  print('df_final is complete!')
  
  mkt_time = (df_final['Date'].dt.time >= datetime.time(9,30,00)) & (df_final['Date'].dt.time <= datetime.time(16,00,00))
  df_final['MktT'] = 'Close'
  df_final.loc[mkt_time, 'MktT'] = 'Open'
  tk = df_final['Ticker'][0]
  opendt = []
  embedded = []
  void_arr = []
  for y in range(df_final.shape[0]):
    if df_final.loc[y,'MktT'] == 'Close':
      void_arr.append(df_final.loc[y, np.arange(0,100)])
    else:
      opendt.append(df_final.loc[y, 'Date'])
      void_arr.append(df_final.loc[y, np.arange(0,100)])
      embedded.append(np.mean(void_arr, axis = 0))
      void_arr = []

  new_df = pd.DataFrame({'Ticker': tk, 'Date': opendt})
  new_df = pd.concat([new_df, pd.DataFrame(embedded)], axis=1)


  print('new_df is complete!')

  filename_csv = csv_set[lp]
  price_df = pd.read_csv(os.path.join('data', filename_csv), usecols=range(7))
  price_df['Date'] = pd.to_datetime(price_df['Date'], format='%d-%b-%Y %H:%M').dt.tz_localize('US/Eastern')
  new_df = new_df.drop(['Ticker'], axis=1)
  new_df = new_df.set_index(['Date'])
  price_df = price_df.set_index(['Date'])
  times = []
  for ix in price_df.index:
    if ix in new_df.index:
      times.append(ix)
  full_df = price_df.join(new_df).loc[times[0]:times[-1]].fillna(method='ffill')

  print('full_df is complete!')
  full_df.to_pickle(tk + '.pkl')
  print('Uploaded '+ tk + ' data!')
  print('==================================================')


Stemming and tokenizing is complete!
df_final is complete!
new_df is complete!
full_df is complete!
Uploaded DIS data!
